In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
os.environ["CUDA_LAUNCH_BLOCKING"]

'1'

In [4]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('CUDA version:', torch.version.cuda)
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 960
CUDA version: 11.7
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import MaskFormerFeatureExtractor, MaskFormerForInstanceSegmentation
import PIL
from PIL import Image
import requests
from transformers import pipeline
import datasets
from datasets import load_dataset
import os
import evaluate
import torch
import cv2
import json
import codecs

c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
image_folder = './WE3DS/images/'
annotation_folder = './WE3DS/annotations/segmentation/SegmentationLabel/'
# Define the paths to the images and annotations
all_image_names = np.array(os.listdir(image_folder))

In [7]:
plant_classification = {
    'void': 'void',
    'soil': 'soil',
    'broad bean': 'crop',
    'corn spurry': 'weed',
    'red-root amaranth': 'weed',
    'common buckwheat': 'crop',
    'pea': 'crop',
    'red fingergrass': 'weed',
    'common wild oat': 'weed',
    'cornflower': 'weed',
    'corn cockle': 'weed',
    'corn': 'crop',
    'milk thistle': 'weed',
    'rye brome': 'weed',
    'soybean': 'crop',
    'sunflower': 'crop',
    'narrow-leaved plantain': 'weed',
    'small-flower geranium': 'weed',
    'sugar beet': 'crop'
}

In [8]:
def get_image_meta_filepath(plant_name):
    return './meta/' + plant_name + '_images.json'

In [9]:
def get_image_list_for_crop(crop_name):
    # Create an empty list to store the dataset
    image_list = []
    crop_image_names = json.load(codecs.open(get_image_meta_filepath(crop_name), 'r', 'utf-8-sig'))
    weed_image_names = json.load(codecs.open(get_image_meta_filepath('no_crop'), 'r', 'utf-8-sig'))
    image_names = crop_image_names + weed_image_names

    # Iterate over the image and annotation paths
    for image_name in crop_image_names:
        # Load the image and annotation using PIL
        image = Image.open(image_folder + image_name)
        annotation = Image.open(annotation_folder + image_name)
        
        # Create a dictionary entry for the dataseta
        entry = {'image': image, 'annotation': annotation}
        
        # Add the entry to the dataset
        image_list.append(entry)

    return image_list

In [10]:
def create_and_split_dataset_for_crop(crop_image_list):
    dataset = datasets.Dataset.from_list(crop_image_list)
    dataset = dataset.train_test_split(test_size=0.5)
    train_ds = dataset["train"]
    val_ds, test_ds = dataset["test"].train_test_split(test_size=0.5).values()
    return train_ds, val_ds, test_ds

In [11]:
broad_bean_image_list = get_image_list_for_crop('broad_bean')
broad_bean_train_ds, broad_bean_val_ds, broad_bean_test_ds = create_and_split_dataset_for_crop(broad_bean_image_list)

In [12]:
print("Training subset number of images: " + str(broad_bean_train_ds.num_rows))
print("Validation subset number of images: " + str(broad_bean_val_ds.num_rows))
print("Test subset number of images: " + str(broad_bean_test_ds.num_rows))

Training subset number of images: 105
Validation subset number of images: 52
Test subset number of images: 53


In [13]:
from transformers import AutoImageProcessor

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, reduce_labels=True)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [14]:
def train_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs

In [15]:
metric = evaluate.load("mean_iou")

In [16]:
with open('./WE3DS/class_names.txt', 'r') as file:
    labels = [line.strip() for line in file]

ids = list(range(1, 20))

id2label = dict(zip(ids, labels))
label2id = dict(zip(labels, ids))

num_labels = len(labels)

print("id2label:", id2label)
print("label2id:", label2id)

id2label: {1: 'void', 2: 'soil', 3: 'broad bean', 4: 'corn spurry', 5: 'red-root amaranth', 6: 'common buckwheat', 7: 'pea', 8: 'red fingergrass', 9: 'common wild oat', 10: 'cornflower', 11: 'corn cockle', 12: 'corn', 13: 'milk thistle', 14: 'rye brome', 15: 'soybean', 16: 'sunflower', 17: 'narrow-leaved plantain', 18: 'small-flower geranium', 19: 'sugar beet'}
label2id: {'void': 1, 'soil': 2, 'broad bean': 3, 'corn spurry': 4, 'red-root amaranth': 5, 'common buckwheat': 6, 'pea': 7, 'red fingergrass': 8, 'common wild oat': 9, 'cornflower': 10, 'corn cockle': 11, 'corn': 12, 'milk thistle': 13, 'rye brome': 14, 'soybean': 15, 'sunflower': 16, 'narrow-leaved plantain': 17, 'small-flower geranium': 18, 'sugar beet': 19}


In [17]:
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [18]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.running_mean', 'decode_head.linear_c.1.proj.weight', 'decode_head.classifier.weight', 'decode_head.linear_c.3.proj.weight', 'decode_head.classifier.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_fuse.weight', 'decode_head.batch_norm.bias', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.0.proj.weight', 'decode_head.batch_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=100,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    seed=12,
)

In [20]:
def initialize_trainer(train_ds, test_ds) :
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    return trainer

In [21]:
broad_bean_train_ds.set_transform(train_transforms)
broad_bean_val_ds.set_transform(train_transforms)
broad_bean_test_ds.set_transform(train_transforms)

In [22]:
braod_bean_trainer = initialize_trainer(broad_bean_train_ds, broad_bean_val_ds)
braod_bean_trainer.train()

c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 1/2100 [00:07<4:15:20,  7.30s/it]

{'loss': 3.0074, 'learning_rate': 5.997142857142857e-05, 'epoch': 0.05}


  0%|          | 2/2100 [00:13<3:59:17,  6.84s/it]

{'loss': 2.9853, 'learning_rate': 5.994285714285715e-05, 'epoch': 0.1}


  0%|          | 3/2100 [00:19<3:47:56,  6.52s/it]

...

C:\Users\Kate\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9\mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
C:\Users\Kate\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9\mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
                                                    
 12%|█▏        | 260/2100 [32:45<1:58:12,  3.85s/it]

{'eval_loss': 0.9084311723709106, 'eval_mean_iou': 0.7547839265485752, 'eval_mean_accuracy': 0.9845865597475816, 'eval_overall_accuracy': 0.9805835236740404, 'eval_per_category_iou': [0.9801499169315913, 0.5294179361655591, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'eval_per_category_accuracy': [0.9803984768467761, 0.9887746426483872, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'eval_runtime': 70.9842, 'eval_samples_per_second': 0.733, 'eval_steps_per_second': 0.155, 'epoch': 12.38}


 12%|█▏        | 260/2100 [32:45<3:51:50,  7.56s/it]

{'train_runtime': 1965.6033, 'train_samples_per_second': 5.342, 'train_steps_per_second': 1.068, 'train_loss': 1.5818912391479198, 'epoch': 12.38}


TrainOutput(global_step=260, training_loss=1.5818912391479198, metrics={'train_runtime': 1965.6033, 'train_samples_per_second': 5.342, 'train_steps_per_second': 1.068, 'train_loss': 1.5818912391479198, 'epoch': 12.38})

In [23]:
braod_bean_trainer.state.log_history

[{'loss': 3.0074,
  'learning_rate': 5.997142857142857e-05,
  'epoch': 0.05,
  'step': 1},
 {'loss': 2.9853,
  'learning_rate': 5.994285714285715e-05,
  'epoch': 0.1,
  'step': 2},
 {'loss': 2.9659,
  'learning_rate': 5.9914285714285716e-05,
  'epoch': 0.14,
  'step': 3},
 {'loss': 2.89,
  'learning_rate': 5.988571428571429e-05,
  'epoch': 0.19,
  'step': 4},
 {'loss': 2.9191,
  'learning_rate': 5.9857142857142856e-05,
  'epoch': 0.24,
  'step': 5},
 {'loss': 2.8563,
  'learning_rate': 5.982857142857143e-05,
  'epoch': 0.29,
  'step': 6},
 {'loss': 2.8623,
  'learning_rate': 5.9800000000000003e-05,
  'epoch': 0.33,
  'step': 7},
 {'loss': 2.784,
  'learning_rate': 5.977142857142857e-05,
  'epoch': 0.38,
  'step': 8},
 {'loss': 2.789,
  'learning_rate': 5.9742857142857144e-05,
  'epoch': 0.43,
  'step': 9},
 {'loss': 2.7765,
  'learning_rate': 5.971428571428572e-05,
  'epoch': 0.48,
  'step': 10},
 {'loss': 2.7413,
  'learning_rate': 5.9685714285714284e-05,
  'epoch': 0.52,
  'step': 11

In [24]:
braod_bean_trainer.save_model('models/broad_bean')
# broad_bean_test_ds

In [25]:
# Interpret results
# Add method to rewrite labels in the annotions